In [3]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase
from datasets import load_from_disk
from ner import NER
from path_retriever import PathRetriever

load_dotenv('db.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [4]:
# DATASET_NAME = 'prime'
DATASET_NAME = 'mag'

In [5]:
# All data needed
qa = load_from_disk(f'{DATASET_NAME}-data/qa')

In [ ]:
# Entity matching on all data
with GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    ner = NER(dataset_name=DATASET_NAME, driver=driver)
    qa_with_ner = qa['train'] \
        .map(lambda x: x | {'predicted_entities' : ner.find_source_nodes(x['question'], driver=driver, openai_api_key=OPENAI_API_KEY)}, num_proc=8)
qa_with_ner.save_to_disk(f'{DATASET_NAME}-data/qa_with_ner/train')

Parameter 'function'=<function <lambda> at 0x383fbcea0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map (num_proc=8):   0%|          | 0/7993 [00:00<?, ? examples/s]

Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Transaction failed and will be retried in 0.9752178641859097s (Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687))))
Transaction fail

In [10]:
qa_with_ner = load_from_disk(f'{DATASET_NAME}-data/qa_with_ner')

with GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    path_retriever = PathRetriever(dataset_name=DATASET_NAME)
    
    qa_with_cypher_queries = qa_with_ner \
        .map(lambda x: x | path_retriever.retrieve_paths(driver=driver, src_names=x['predicted_entities'], tgt_ids=x['answer_ids']), num_proc=8)
    
qa_with_cypher_queries.save_to_disk(f'{DATASET_NAME}-data/qa_with_cypher_queries/')

Map (num_proc=8):   0%|          | 0/100 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/2665 [00:00<?, ? examples/s]

TimeoutError: 